# Neural Networks For Feature Selection

methodology
>implement pattern-matching algorithm to group data by same piece of information <br>
>see what features to engineer<br>
>What features do we need for the immediacy label?<br>
>See how to deal with those missing values<br>
>feed 5 features at a time to neural network. See the accuracy.<br>

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
NHAMCS = pd.read_sas(filepath_or_buffer = 'NHAMCS.sas7bdat')

In [ ]:
NHAMCS.head()

In [ ]:
NHAMCS.shape

### NaNs Exploration

>In this subsection of the data quality section, we explore the following:
- how many NaNs are there?
- how can we visualize them?
- what could be the reasons for these NaNs?
- further steps to be taken for data quality remarks.

### How many NaNs are there?

Count the number of NaNs Across all columns.

In [ ]:
nans = NHAMCS.isnull().sum()

Order the series descendingly to be able to see those features with the most NaNs. These will most likely be eliminated from the dataset.

In [ ]:
nans.sort_values(ascending = False, inplace = True)

Make sure all the features are captured.

In [ ]:
nans.count() 

Let's have a quick look...

In [ ]:
nans 

We see that the first entries seem to have ALL NaNs! How prevalent is this? Let's see.

In [ ]:
nans_max = nans[nans >= int(0.99*16709)]

In [ ]:
nans_max.count()

In [ ]:
448/949

47% of the features are almost all NaNs! Let's continue to investigate different % of missing values. We exclude the ones with almost all NaNs.

In [ ]:
nans_no_max = nans[nans < int(0.99*16709)]

In [ ]:
nans_90 = nans_no_max[nans_no_max >= int((0.90*16709))]

In [ ]:
print(nans_90.count())
print(104+448, 'features have 90%+ NaNs. That is', 552/949 *100, 'of the features.', 949-448, 'of 949 features have less than 90% NaN.')

What about those with 70% NaNs? (excluding the ones with 90%+ NaNs)

In [ ]:
nans_70 = nans_no_max[nans_no_max > int((0.7*16709))]
nans_70.count()
print(134+552,'features have 70%+ NaNs. That is', 686/949*100,'of the features.',949-686, 'of 949 features have less than 70%+ NaNs.' )

Some interesting findings. Let's visualize the NaN trend, descendingly.

### NaNs Trend Visualization

In [ ]:
nans.plot(kind = 'bar', figsize = (10,10), title = 'Number of NaNs per Column', xticks = []);

### Why is so much data unavailable?

>From the plot, it seems like we have approximately 550 features whose missing values exceed 80%. Looking at the dataset, this is not exactly surprising. There are four mains reasons. 

#### 1. Information that could have concisely been represented is represented in multiple variables.


>##### EXAMPLE: Payment method. We could use feature engineering to just represent what the source of payment was.
><ol>
    <li>PAYPRIV = "Expected source(s) of payment for this visit: Private insurance"</li>
    <li>PAYMCARE ="Expected source(s) of payment for this visit: Medicare"</li>
    <li>PAYMCAID = "Expected source(s) of payment for this visit: Medicaid or CHIP or other state-based program"</li>
    <li>PAYWKCMP = "Expected source(s) of payment for this visit: Workers' compensation"</li>
    <li>PAYSELF = "Expected source(s) of payment for this visit: Self pay"</li>
    <li>PAYNOCHG = "Expected source(s) of payment for this visit: No charge/Charity"</li>
    <li>PAYOTH = "Expected source(s) of payment for this visit: Other"</li>
    <li>PAYDK = "Expected source(s) of payment for this visit: Unknown"</li>
    <li>PAYTYPER = "Recoded primary expected source of payment for this visit (based on hierarchy)"</li>
</ol>


#### 2. Some features, collectively, are a "list" for the same piece of information.


>##### EXAMPLE: Paitent's medications. Not all pateints are prescribed 30 medicines.
><ol>
    <li>MED1="Medication #1"</li>
    <li>MED2="Medication #2"</li>
    <li>MED3="Medication #3"</li>
    <li>MED4="Medication #4"</li>
    <li>...</li>
    <li>MED27="Medication #27"</li>
    <li>MED28="Medication #28"</li>
    <li>MED29="Medication #29"</li>
    <li>MED30="Medication #30"</li>
</ol>

#### 3. With or without reason 2, a collection of features represent a level of detail for the same information.

>##### EXAMPLE: Paitent's complaints. Along with the listing, there is a "level of detail" to the complaint.
><ol>
    <li>RFV1 = "Patient's complaint, symptom, or other reason for visit #1 - detailed category"</li>
    <li>RFV1 = "Patient's complaint, symptom, or other reason for visit #2 - detailed category"</li>
    <li>RFV1 = "Patient's complaint, symptom, or other reason for visit #3 - detailed category"</li>
    <li>RFV1 = "Patient's complaint, symptom, or other reason for visit #4 - detailed category"</li>
    <li>RFV1 = "Patient's complaint, symptom, or other reason for visit #5 - detailed category"</li>
    <li>...</li>
    <li>RFV13D = "Patient's complaint, symptom, or other reason for visit #1 - detailed category"</li>
    <li>RFV13D = "Patient's complaint, symptom, or other reason for visit #2 - detailed category"</li>
    <li>RFV13D = "Patient's complaint, symptom, or other reason for visit #3 - detailed category"</li>
    <li>RFV13D = "Patient's complaint, symptom, or other reason for visit #4 - detailed category"</li>
    <li>RFV13D = "Patient's complaint, symptom, or other reason for visit #5 - detailed category"</li>
</ol>


#### 4. There are features that are reserved for only special pateints. 

>##### EXAMPLE: 'AGEDAYS' - age days for babies. This could be missing for the rest of the sample.
><ol>
    <li>AGE="Patient age in years"</li>
    <li>AGER="Age recode"</li>
    <li>AGEDAYS="Age in days for patients less than one year"</li>
</ol>




#### What are some key takeaways from these observations?

- All the reasons portray how well-suited this dataset is for a myriad of specific use cases and research, albeit with challenging and elongated data cleaning procedures. 

- Reasons 1 to 3 Feature engineering may also be used extensively to extract the essence of the required data for the problem at hand.

- Reason 4 unleashes an extremely important fact: Do not assume that, if 95% of a column's values are missing, that we could safely drop it. It might be catered for a specific use case and hence missing for the majority. For example, if the use case is to study which diseases result in ER visits for babies the most, although this feature could be mostly missing, it is actually important and the missing % is misleading.

- All reasons show us that we might not necessarily need to drop a column with high % of NaNs. From the special features of the available subset along with some domain research, we might be able to replace the missing data accurately.

### Further Comments and Prospective Steps

#### Missing Values Further Steps
- Investigate for also these values: [-9, -8, -7, 0]. -9 always means missing values, so we deal with that as such. However, -8 means "unknown" and -7 means "N/A". 0 may also mean "blank." For features extracted, according to the nature of each feature, we must decide what to do with entries with those values. Some examples include the below.

		
>**1. VALUE RESIDF**<br>
  >  -9='Blank'<br>
   > -8='Unknown'<br>
	1='Private residence'<br>
	2='Nursing home'<br>
	3='Homeless/homeless shelter'<br>
	4='Other'<br>
    
>**2. VALUE WAITTIMEF**<br>
    -9 ='Blank'<br>
    -7 ='Not Applicable'<br>
    
>**3. VALUE PAYTYPERF**<br>
      -9 ='All sources of payment are blank'<br>
      -8 ='Unknown'<br>
       1 ='Private insurance'<br>
       2 ='Medicare'<br>
       3 ='Medicaid or CHIP or other state-based program'<br>
       4 ="Worker's compensation"<br>
       5 ='Self-pay'<br>
       6 ='No charge/Charity'<br>
       7 ='Other'   <br>
 


<br>


- for the extracted features, based on the problem domain, available data %, and relationships with other features, how can we best replace those missing values? Extensive research is needed.


#### Data Quality Concerns For Further Investigation
- are the values consistent?
- are the types suitable?
- is te data too granular or too specific for a use case?
- is the data comprehensive for a use case?
- invesigate the data collection time and determine its relevance for today.
- is the data precise?
- are the features selected complete?



#### MVP: Drop ALL with any NaNs!

In [ ]:
MVP = NHAMCS.dropna(axis='columns')

In [ ]:
MVP.shape

### Target Feature & Its Meaning

IMMEDR="Immediacy with which patient should be seen (unimputed)" => missing values not replaced.

-    -9='Blank'
-    -8='Unknown'
-    0='No triage for this visit but ESA does conduct triage'
-	1='Immediate'
-	2='Emergent'
-	3='Urgent'
-	4='Semi-urgent'
-	5='Nonurgent'
-	7='Visit occurred in ESA that does not conduct nursing triage'

In [ ]:
MVP['IMMEDR'].value_counts()

In [ ]:
MVP['IMMEDR'].isna().sum()

In [ ]:
MVP['IMMEDR'].hist(bins = 17)

In [ ]:
Explore8 = MVP[MVP['IMMEDR']==-8]
Explore8['WAITTIME'].hist(bins=80)

In [ ]:
MVP['WAITTIME'].hist(bins=80)

### Extracting Patterns

### STEP 1: Preparing the Functions

#### HELPER: Function to Extract the Prefixes

In [ ]:
temp = NHAMCS[['RACEUN','RACER','RACERETH','NOPAY']]

In [ ]:
#TESTS PASSED

#n is the length of the prefix
#df is the dataframe
def extract_prefixes(n, df):
    l = []
    for col in df.columns:
        l.append((col[:n]))
    return l

#### HELPER: Find the Longest Prefix

In [ ]:
#TESTS PASSED 
def longest_prefix(df):
    l = []
    for col in df.columns:
        l.append(len(col))
    return max(l)

In [ ]:
longest_prefix(MVP)

This means we stop prefixing at 12.

#### Find Prefix frequency per Prefix Length 

In [ ]:
from collections import Counter

def prefix_freq(n, df):
    l = extract_prefixes(n,df)
    counter=Counter(l)
    counter = {val[0] : val[1] for val in sorted(counter.items(), key = lambda x: (-x[1], x[0]))}
    return counter
#result = [list(counter.keys()),list(counter.values())]


In [ ]:
from collections import Counter

def prefix_freq_no1(n, df):
    final_dict = {}
    l = extract_prefixes(n,df)
    counter=Counter(l)
    
    for key in counter:
        if (counter[key] != 1):
            final_dict[key] = counter[key] 
            final_dict = {val[0] : val[1] for val in sorted(final_dict.items(), key = lambda x: (-x[1], x[0]))}
    return final_dict
#result = [list(counter.keys()),list(counter.values())]


#### Return Features with the Same Prefix

In [ ]:
def pattern_match(prefix, df):
    l = []
    for col in df.columns:
        if(col.startswith(prefix)):
            l.append(col)
    return l

#### Remove Features with a Certain Prefix

In [ ]:
def remove_pref(pref_list, df):
    for pref in pref_list:
        matching = pattern_match(pref,df)
        for col in matching:
            df = df.drop([col], axis = 1)
    return df

### STEP 2: Investigate Prefixes And Their Meanings.

#### For a 9-length prefix...

- PTONLINEE: these variables discuss a pateint's ability to enter or receive information online. Irrelevant to level of immediacy.
- EDISCHSRE: no documentation available. Eliminate.

In [ ]:
prefs = list(prefix_freq_no1(9,MVP).keys())
print(prefs)
MVP = remove_pref(prefs, MVP)
print(MVP.shape)

#### For an 8-length prefix...
- ESUMCSRE: not documented. Remove.

In [ ]:
prefs = (prefix_freq_no1(8,MVP).keys())
print(prefs)
MVP = remove_pref(prefs, MVP)
print(MVP.shape)

#### For a 7-length prefix...

In [ ]:
prefs = (prefix_freq_no1(7,MVP).keys())
print(prefs)

### Understanding Data Types and Encoding

In [ ]:
def check_objects():
    types = MVP.columns.to_series().groupby(MVP.dtypes).groups
    types_dict = {k.name: v for k, v in types.items()}
    print(types_dict.keys())
    print(types_dict['object'])
    
check_objects()

Which ones are worth keeping?

In [ ]:
meds = MVP[['MED1', 'MED2', 'MED3', 'MED4', 'MED5', 'MED6',
       'MED7', 'MED8', 'MED9', 'MED10', 'MED11', 'MED12', 'MED13', 'MED14',
       'MED15', 'MED16', 'MED17', 'MED18', 'MED19', 'MED20', 'MED21', 'MED22',
       'MED23', 'MED24', 'MED25', 'MED26', 'MED27', 'MED28', 'MED29', 'MED30']]

meds


In [ ]:
#for i in range(30):
#    varname = 'MED' + str(i+1)
#    MVP[varname] =  MVP[varname].apply(lambda x: int(x.decode()))

In [ ]:
#mapper = {'M':0,'F':1}
#df_appointments.replace({'Gender':mapper}, inplace = True)
#df_appointments['DaysLeft'] = timedelta_days_left.apply(lambda x: float(x.days))

In [ ]:
meds = MVP[['MED1', 'MED2', 'MED3', 'MED4', 'MED5', 'MED6',
       'MED7', 'MED8', 'MED9', 'MED10', 'MED11', 'MED12', 'MED13', 'MED14',
       'MED15', 'MED16', 'MED17', 'MED18', 'MED19', 'MED20', 'MED21', 'MED22',
       'MED23', 'MED24', 'MED25', 'MED26', 'MED27', 'MED28', 'MED29', 'MED30']]

meds

In [ ]:
#for med in meds.columns:
#    print(meds[meds[med]==-9][med].value_counts()/16709*100)

In [ ]:
#for i in range(24):
#    dropcol = 'MED' + str(i+7)
#    MVP.drop(dropcol, axis = 'columns', inplace = True)
#MVP.shape

For MED7+, 93%+ is missing: drop!

In [ ]:
check_objects()

In [ ]:
causes = MVP[['CAUSE1', 'CAUSE2', 'CAUSE3']]
causes.head()

In [ ]:
check_objects()

In [ ]:
diag = MVP[['ARRTIME', 'CAUSE1', 'CAUSE2', 'CAUSE3', 'DIAG1', 'DIAG2', 'DIAG3',
       'DIAG4', 'DIAG5', 'HDDIAG1', 'HDDIAG2', 'HDDIAG3', 'HDDIAG4',
       'HDDIAG5']]
diag.head()

In [ ]:
bb = b'W230'
bb.decode(encoding='UTF-8',errors='strict')

In [ ]:
for i in range(3):
    varname = 'CAUSE' + str(i+1)
    MVP[varname] =  MVP[varname].apply(lambda x: transform_byte(x.decode()))

### Using Neural Nets to Find The Most Influential Features

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

In [ ]:
#importing libraries for modelling
import tensorflow
from tensorflow.keras.models import Sequential #this is the "base" model. It allows us to create any deep learning architecture of our choice by adding layers to it.
from tensorflow.keras.layers import Activation, Dense #this helps us to create hidden layers
from tensorflow.keras.layers import Dropout #regularization technique to prevent overfitting. It deactivates some neurons at random (weight = 0) to accomplish that.
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
layers = [(20,), (40, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [ ]:
grid_result = grid.fit(X_train_std, Y_train)

In [ ]:
pred_y = grid.predict(X_test_std)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(Y_test,y_pred)